In [ ]:
# pip install optuna numpy pandas pyarrow arrow scikit-optimize quantstats matplotlib ta -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import warnings
from functools import lru_cache

from skopt import gp_minimize
from skopt.plots import plot_evaluations, plot_objective, plot_convergence
from skopt.space import Integer, Categorical
from skopt.utils import use_named_args

from more_factor_test_origin_code import cal_cagr

warnings.filterwarnings('ignore')  # 忽略警告
import pandas as pd

pd.set_option('display.max_columns', None)  # 当列太多时不换行
df = pd.read_parquet('cb_data.pq')  # 导入转债数据
index = pd.read_parquet('index.pq')  # 导入指数数据

In [ ]:
# df.head()

In [ ]:
# 基础设置
start_date = '20220801'  # 开始日期
end_date = '20240325' # 结束日期

In [ ]:
# 参数空间定义
factors = ['pre_close', 'open', 'high', 'low', 'close', 'pct_chg', 'vol',#7
           'amount', 'volatility_stk','mod_conv_prem','remain_cap','conv_prem',#12
           'turnover','theory_value','amount','option_value','dblow',#17
           'theory_bias','ytm','cap_mv_rate','pure_value','bond_prem',#22
           'remain_size','theory_conv_prem','pb','pe_ttm','ps_ttm']#27
# 定义优化空间
space = [
    # 因子编号，为了简单起见，这里假设最多有50个因子，你可以根据实际情况调整
    Integer(0, len(factors) - 1, name='factor1_id'),
    Categorical([1, 2, 3, 4, 5], name='factor1_weight'),
    Categorical([True, False], name='factor1_ascending'),

    Integer(0, len(factors) - 1, name='factor2_id'),
    Categorical([1, 2, 3, 4, 5], name='factor2_weight'),
    Categorical([True, False], name='factor2_ascending'),

    Integer(0, len(factors) - 1, name='factor3_id'),
    Categorical([1, 2, 3, 4, 5], name='factor3_weight'),
    Categorical([True, False], name='factor3_ascending'),
    
    Integer(0, len(factors) - 1, name='factor4_id'),
    Categorical([1, 2, 3, 4, 5], name='factor4_weight'),
    Categorical([True, False], name='factor4_ascending'),
]

In [ ]:
@lru_cache(maxsize=None)
def cached_objective(factor1_id, factor2_id, factor3_id, factor1_weight, factor2_weight, factor3_weight, factor1_ascending, factor2_ascending, factor3_ascending):
    # 提取因子ID并确保它们是唯一的
    factor_ids = [factor1_id, factor2_id, factor3_id]
    if len(set(factor_ids)) < 3:
        return 1e6  # 假设最大化问题，返回一个很大的负值作为惩罚
    rank_factors = []
    params = locals()
    for i in range(1, 4):
        factor_info = {
            'name': factors[factor_ids[i - 1]],
            'weight': params[f'factor{i}_weight'],
            'ascending': params[f'factor{i}_ascending']
        }
        rank_factors.append(factor_info)
    cagr = -cal_cagr(df, start_date, end_date, rank_factors)
    print("当前迭代的rank_factors:", rank_factors, ", cagr:", cagr)
    return cagr

In [ ]:
# 定义一个wrapper函数来调用你的回测函数，以适配贝叶斯优化过程
@use_named_args(space)
def objective(**params):
    return cached_objective(
        params['factor1_id'], params['factor2_id'], params['factor3_id'],
        params['factor1_weight'], params['factor2_weight'], params['factor3_weight'],
        params['factor1_ascending'], params['factor2_ascending'], params['factor3_ascending']
    )

In [ ]:
# 运行贝叶斯优化
res = gp_minimize(objective, space, n_calls=1000, random_state=1212, n_initial_points=50, n_jobs=-1)
# 打印最优参数
print("最优参数：", res.x)
print("最优参数下的目标函数值：", -res.fun)
print(f"name:{factors[res.x[0]]}, weight:{res.x[1]}, ascending:{res.x[2]}")
print(f"name:{factors[res.x[3]]}, weight:{res.x[4]}, ascending:{res.x[5]}")
print(f"name:{factors[res.x[6]]}, weight:{res.x[7]}, ascending:{res.x[8]}")
print(f"name:{factors[res.x[9]]}, weight:{res.x[10]}, ascending:{res.x[11]}")